# Modelo de Lead Scoring

En este notebook iremos desarrollando las tareas necesarias para construir un modelo de lead scoring comenzando desde la ingesta de datos

## 1. Data analysis

Importamos todas las librerias necesarias y consumimos los datos

In [ ]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam


warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize':(21.7,21.27)})
pd.options.display.max_columns = None
data = pd.read_csv('./leads/Leads.csv')

### 1.1 Data Cleaning

En esta etapa nos familiarizamos con los datos e intentamos entender que tipo de informacion aporta cada variable. De aqui surgiran hipotesis respecto a que variables son mas relevantes y que tipo de limpieza/curacion sera necesaria

Empezamos desplegando las primeras 5 filas para entender que cosas trae

In [ ]:
data.head()

In [ ]:
data.describe()

Notamos:

- Hay algunos campos con valores nulos
- Hay un campo "booleano" (converted), que es el que probablemente querramos predecir
- Empezamos a intuir los valores que puede tomar cada columna

Ahora nos falta entender mejor los campos no numericos


In [ ]:
data.describe(include=[object])

Notamos:

- Hay campos identificadores que podriamos excluir del modelo (`prospect id`, `lead Number`)
- Hay campos que unicamente tienen un valor (`Magazine` ,`I agree to pay the amount through cheque`, `Receive more updates` , etc) que podriamos excluir por no agregar info
- Hay algunos campos que podriamos procesar un poco para que representen mejor su informacion (Campos de `Asymmetrique`)
- Hay mas campos "booleanos" codificados como texto (`Do not Email`, `Do not call`)

Ya que notamos que hay valores nulos, veamos cuantos hay en cada columna


In [ ]:
data.isnull().sum()

Vemos que hay muchas columnas con valores nulos. Y muchos de ellas con casi el 50% de sus valores asi. Vamos a deshacernos de ellas y de las otras columnas que identificamos y a convertir las columnas booleanas a numerica


In [ ]:
data = data.dropna(axis=1)
data = data.drop(columns=['Prospect ID', 'Lead Number', 'I agree to pay the amount through cheque', 'Magazine', 'Receive More Updates About Our Courses', 
                          'Update me on Supply Chain Content', 'Get updates on DM Content' ])
string_to_boolean_map = {"Yes": 1, "No": 0}
for column in data.columns:
    if 'Yes' in data[column].values and 'No' in data[column].values:
        data[column] = data[column].map(string_to_boolean_map)


In [ ]:
data

Luego de haber *limpiado* nuestros datos, estamos en conndiciones de comenzar la **exploracion** en busqueda de relaciones interesantes entre nuestras variables

### 1.2 Exploratory Analysis

Comenzamos graficando algunas de las variables respecto a nuestra variable **a predecir**

In [ ]:
sns.barplot(data=data, y='Converted', x='Lead Origin')

In [ ]:
sns.barplot(data=data, y='Converted', x='Do Not Email')

In [ ]:
sns.barplot(data=data, y='Converted', x='Search')

In [ ]:
sns.barplot(data=data, y='Converted', x='Last Notable Activity')

In [ ]:
sns.histplot(data=data, x='Total Time Spent on Website')

## 2. Model Building

Ahora que hemos explorado los datos y tenemos una mejor idea de como se relacionan con la variable a predecir, comenzaremos a construir modelos tentativos para predecir la conversion

Comenzamos dandole un ultimo pre-procesamiento a nuestros datos antes de que puedan usarse de input para los modelos. Hay que convertir las variables categoricas (`Last Notable Activity` y `Lead Origin`) a algo sobre lo que se pueda operar matematicamente. Utilizaremos **one-hot encoding** para lograrlo. Pandas provee una forma sencilla de aplicar esta transformacion


In [ ]:
one_hot_encoded_variables = pd.get_dummies(data[['Lead Origin', 'Last Notable Activity']], drop_first=True)
data = pd.concat([data, one_hot_encoded_variables], axis=1)
data = data.drop(['Lead Origin', 'Last Notable Activity'], axis=1)

In [ ]:
one_hot_encoded_variables


### 2.1 Creacion de datasets de entrenamiento y de prueba

Ahora debemos separar la **variable a predecir** del resto de los **predictores**, asi como separar nuestros datos en datasets de **entrenamiento** y **prueba** 

In [ ]:
predictors = data.drop(['Converted'], axis=1)
to_predict = data['Converted'] 
X_train, X_test, y_train, y_test = train_test_split(predictors, to_predict, train_size=0.8, test_size=0.2)

### 2.2 Feature Scaling

Todas las variables predictoras, a excepcion del tiempo total de visita al sitio web, son booleanas (sus valores pueden ser 0 o 1). Para que el modelo no se sesgue con esa otra variable, conviene aplicar algun tipo de escalado (en nuestro caso, **estandarizacion**) sobre esa variable para convertir sus valores a una unidad "estandar" compartida por todas las demas (en este caso, la "desviacion estandar")


In [ ]:
scaler = StandardScaler()
X_train['Total Time Spent on Website'] = scaler.fit_transform(X_train[['Total Time Spent on Website']])
X_test['Total Time Spent on Website'] = scaler.fit_transform(X_test[['Total Time Spent on Website']])

In [ ]:
X_train

### 2.3 Model Fitting

En esta etapa entrenaremos los diferentes modelos que creemos mas adecuados para el problema



In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_regression_model = LogisticRegression().fit(X_train, y_train)


### 2.4 Evaluacion de la calidad del modelo

La calidad de la estimacion en el dataset de **prueba** es


In [ ]:
logistic_regression_model.score(X_test, y_test)


Y en el de **entrenamiento**

In [ ]:
logistic_regression_model.score(X_train, y_train)

Realizamos la **matriz de confusion** y obtenemos las metricas asociadas a estos valores

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

ConfusionMatrixDisplay(confusion_matrix(y_test, logistic_regression_model.predict(X_test))).plot(cmap="YlOrRd")
print(classification_report(y_test, logistic_regression_model.predict(X_test)))

## Probando una Red neuronal

Como modelo competidor, probamos entenar una red neuronal (Perceptron multicapa) y evaluamos su desempeno respecto a la regresion logistica

In [ ]:
formatted_data = X_train.values
dimensions = formatted_data.shape

adam_optimizer = Adam(learning_rate=0.01)
neural_net = Sequential()
neural_net.add(Dense(128, input_dim=dimensions[1], activation='tanh'))
neural_net.add(Dense(256, input_dim=dimensions[1], activation='tanh'))
neural_net.add(Dense(128, input_dim=dimensions[1], activation='tanh'))
neural_net.add(Dense(1, activation='sigmoid'))
neural_net.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
neural_net.fit(formatted_data, y_train.values, epochs=100, batch_size=1000, validation_split=0.2)


In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test.values, neural_net.predict_classes(X_test.values))).plot(cmap="YlOrRd")
print(classification_report(y_test.values, neural_net.predict_classes(X_test.values)))